# Abalone Age Prediction by Multiclass Classification using ANN

## Load dataset for the training process

In [1]:
import os
import numpy as np
import pandas as pd

from typing import List

### Open the CSV

In [2]:
df_train = pd.read_csv("abalone_train.csv")
df_val = pd.read_csv("abalone_val.csv")
print("Num train:", len(df_train))
print("Num val:", len(df_val))
df_train.head()

Num train: 2924
Num val: 418


,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,1,0.305,0.225,0.070,0.1485,0.0585,0.0335,0.0450,7
1,2,0.475,0.370,0.125,0.5095,0.2165,0.1125,0.1650,9
2,0,0.460,0.370,0.120,0.5335,0.2645,0.1080,0.1345,6
3,2,0.575,0.450,0.155,0.9480,0.4290,0.2060,0.2590,7
4,2,0.505,0.440,0.140,0.8275,0.3415,0.1855,0.2390,8


### Create dataset for training and validation process

In [3]:
import tensorflow as tf

2023-05-06 18:37:23.293931: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-06 18:37:23.299309: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 18:37:23.350478: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 18:37:23.359717: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 18:37:24.349600: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### Features to be used and target

In [4]:
x_names = ["Sex", "Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight"]
y_name = "Rings"
y_classes = list(range(1, 30)) # 1 to 29 (from EDA)

### Convert Pandas dataframe to TF dataset

In [5]:
def df_to_dataset(df: pd.DataFrame) -> tf.data.Dataset:
    features = df[x_names].to_numpy(dtype=np.float32)
    labels = df[[y_name]].to_numpy(dtype=np.int64).squeeze(axis=1)
    labels = np.clip(labels, min(y_classes), max(y_classes)) # limit the age range to 1 to 29
    labels = labels - 1 # map class #0 to Rings 1
    my_dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    return my_dataset

In [6]:
ds_train = df_to_dataset(df_train)
ds_val = df_to_dataset(df_val)

2023-05-06 18:37:25.450323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-06 18:37:25.450624: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Use the dataset

In [7]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 128
ds_train = ds_train.shuffle(SHUFFLE_BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE, drop_remainder=True) # drop last for stability
ds_val = ds_val.batch(BATCH_SIZE)

## Model training

In [8]:
from datetime import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, accuracy_score

In [9]:
MAX_EPOCHS = 2000
INIT_LR = 1e-5

### Which device we will use for training process (CPU/GPU)

If a TensorFlow operation has both CPU and GPU implementations, by default, the GPU device is prioritized when the operation is assigned. So, no need to manually specify here.

*Reference: [Use a GPU](https://www.tensorflow.org/guide/gpu)*

In [10]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Create the model

In [11]:
import nets

model = nets.MLP(n_classes=len(y_classes))

### Define the loss function and the optimizer

In [12]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=INIT_LR)

### Prepare the logger

In [13]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = os.path.join('runs_clf', 'train_{}'.format(timestamp))
os.makedirs(save_dir, exist_ok=True)
writer = tf.summary.create_file_writer(save_dir)
writer.set_as_default()

### The training and validation process

During the training process, launch tensorboard to see the logged train/val metrics
```bash
tensorboard --logdir runs_clf
```
Then, open the link using web browser

In [14]:
# Variables to hold some training status
epoch_number = 0
lowest_loss = np.inf
best_f1 = 0.
# Training loop
for epoch in tqdm(range(MAX_EPOCHS)):
    running_loss = 0.
    for i, data in enumerate(ds_train):
        # Every data instance is an input & label pair
        inputs, labels = data
        # Create GradientTape for recording operations for automatic differentiation
        with tf.GradientTape() as tape:
            # Make predictions for this batch
            # training=True is only needed if there are layers with different
            # behavior during training versus inference (e.g. Dropout).
            outputs = model(inputs, training=True)
            # Compute the loss
            loss = loss_fn(labels, outputs)
        # Compute the gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        # Adjust learning weights
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        # Gather data and report
        running_loss += loss.numpy()
    # Calculate the average training loss
    avg_loss = running_loss / (i + 1)

    # Model validation part
    running_vloss = 0.0
    y_true = []
    y_pred = []
    for i, vdata in enumerate(ds_val):
        vinputs, vlabels = vdata
        y_true.extend(vlabels.numpy().tolist())
        # We don't need gradients for the model validation process
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        voutputs = model(vinputs, training=False)
        vloss = loss_fn(vlabels, voutputs)
        running_vloss += vloss.numpy()
        # the class with the highest energy is what we choose as prediction
        predicted = tf.math.argmax(voutputs, axis=1)
        y_pred.extend(predicted.numpy().tolist())

    # Calculate the average validation loss
    avg_vloss = running_vloss / (i + 1)
    # Calculate our classification metrics
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="weighted")

    # Log the running loss averaged per batch
    # for both training and validation
    tf.summary.scalar('train/loss', avg_loss, epoch_number + 1)
    tf.summary.scalar('val/loss', avg_vloss, epoch_number + 1)
    tf.summary.scalar('val/acc', acc, epoch_number + 1)
    tf.summary.scalar('val/weighted_f1', f1, epoch_number + 1)
    tf.summary.flush()


    # Track best performance, and save the model's state (weights)
    if f1 > best_f1:
        best_f1 = f1
        model_path = os.path.join(save_dir, 'best')
        model.save_weights(model_path)
    if avg_vloss < lowest_loss:
        lowest_loss = avg_vloss
        model_path = os.path.join(save_dir, 'lowest_loss')
        model.save_weights(model_path)
    model_path = os.path.join(save_dir, 'last')
    model.save_weights(model_path)

    epoch_number += 1

  0%|          | 0/2000 [00:00<?, ?it/s]

2023-05-06 18:37:26.318368: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [2924]
	 [[{{node Placeholder/_1}}]]
2023-05-06 18:37:26.318576: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [2924]
	 [[{{node Placeholder/_1}}]]
2023-05-06 18:37:28.283475: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [418]
	 

## References
[1] https://www.tensorflow.org/tutorials/quickstart/advanced

[2] https://www.tensorflow.org/tensorboard/scalars_and_keras

[3] https://www.tensorflow.org/tutorials/keras/save_and_load
